In [1]:
import json
import pandas as pd
import json
import requests

In [9]:
stock = input("Enter stock ticker you want to investigate: ")
region = input("Enter region you want to investigate: ")
valid_regions = ["US", "AU", "CA", "FR", "DE", "HK", "IT", "ES", "GB", "IN"]

while region not in valid_regions:
    print("Invalid region")
    region = input("Enter region you want to investigate: ")

Enter stock ticker you want to investigate: GLD
Enter region you want to investigate: AU


In [10]:
apikey="a9ycWh2p84aAxtd3io5wQakwbaFdW8lLv2oOC7Dd"
url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":stock}
headers = {
    'x-api-key': apikey
    }
response = requests.request("GET", url, headers=headers, params=querystring)

if response.status_code == 200:
    data = response.json()
    if 'finance' in data and 'result' in data['finance']:
        if data['finance']['result']:
            print("This is a valid stock symbol")
            urlb = "https://yfapi.net/ws/insights/v1/finance/insights"
            querystringb = {"symbol":stock}
            headers = {
              'x-api-key': apikey
                  }
            responseb = requests.request("GET", urlb, headers=headers, params=querystringb)
        else:
            print(stock+ ", this stock may be invalid or missing data.")
            urlb = "https://yfapi.net/ws/insights/v1/finance/insights"
            querystringb = {"symbol":stock}
            headers = {
                'x-api-key': apikey
                  }
            responseb = requests.request("GET", urlb, headers=headers, params=querystringb)
    else:
        print(stock + ", this stock may be missing data")
        urlb = "https://yfapi.net/ws/insights/v1/finance/insights"
        querystringb = {"symbol":stock}
        headers = {
            'x-api-key': apikey
                  }
        responseb = requests.request("GET", urlb, headers=headers, params=querystringb)
else:
    print(stock+ " may not be a valid stock symbol")



urlc = "https://yfapi.net/v1/finance/trending/"+region
headers = {
    'x-api-key': apikey
    }
responsec = requests.request("GET", urlc, headers=headers)


GLD, this stock may be missing data


In [15]:
stock_json = response.json()
stock_jsonb = responseb.json()
region_json = responsec.json()

trending_stocks = []
num_stocks = len(region_json['finance']['result'][0]['quotes'])
for i in range(0,num_stocks):
    trending_stocks.append(region_json['finance']['result'][0]['quotes'][i]['symbol'])


print("The stock ticker is: "+stock)

#Sometimes the display name/longname is not included in what the API gets
stock_name = stock_json['quoteResponse']['result'][0]
full_name = stock_name.get('longName') or stock_name.get('displayName')

if full_name:
    print("The full name of the stock is: " + full_name)
else:
    print("No full name available.")

print("Price: $" + str(stock_json['quoteResponse']['result'][0]["regularMarketPrice"])
+ "\n52 week high: $" + str(stock_json['quoteResponse']['result'][0]["fiftyTwoWeekHigh"])
+ "\n52 week low: $" + str(stock_json['quoteResponse']['result'][0]["fiftyTwoWeekLow"]))


try:
    target_price = stock_jsonb['finance']['result']['instrumentInfo']['recommendation']['targetPrice']
    print("The target price is: $" + str(target_price))
except KeyError:
    print("No target price available.")

print("")
print("Currently trending stock(s) in the "+region+" market are: "+ str(trending_stocks))


The stock ticker is: GLD
The full name of the stock is: SPDR Gold Shares
Price: $245.02
52 week high: $247.37
52 week low: $168.3
No target price available.

Currently trending stocks in the AU market are: ['^AORD']


In [16]:
import pandas as pd

price = stock_json['quoteResponse']['result'][0]["regularMarketPrice"]
high_52_week = stock_json['quoteResponse']['result'][0]["fiftyTwoWeekHigh"]
low_52_week = stock_json['quoteResponse']['result'][0]["fiftyTwoWeekLow"]

try:
    target_price = stock_jsonb['finance']['result']['instrumentInfo']['recommendation']['targetPrice']
except KeyError:
    target_price = "No target price available"

dataframe = {
    "Stock Ticker": [stock],
    "Full Name": [full_name],
    "Price": [price],
    "52 Week High": [high_52_week],
    "52 Week Low": [low_52_week],
    "Target Price": [target_price],
    "Trending Stocks": [', '.join(trending_stocks)]
}

df = pd.DataFrame(dataframe)
df.to_csv("stock_info.csv", index=False)

